In [71]:
#Restrict training to one gpu
import imp
try:
        imp.find_module('setGPU')
        import setGPU
except ImportError:
        found = False
#/////////////////////
import matplotlib
import tensorflow as tf
sess = tf.Session()
import matplotlib.pyplot as plt
import keras.backend as K
K.set_session(sess)
import pylab as P
import pandas as pd
import numpy as np
import keras.callbacks
import glob
import math

#Useful sklearn functions
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

#Keras model related imports
from keras.models import Model
from keras.layers import Input,Dense,Conv1D,LSTM,Flatten,Dropout,Activation
import keras.backend as K
from keras.callbacks import ReduceLROnPlateau
from keras.regularizers import l2
from keras import optimizers
from keras.callbacks import Callback

#For training progress bar to display correctly
#see: https://github.com/bstriner/keras-tqdm/blob/master/examples/keras_progress_bars.ipynb
from keras_tqdm import TQDMNotebookCallback


In [72]:
#/////Reading the data
#/////////////////////
clus = np.genfromtxt("/Users/innocent/data/clusterShape.csv", delimiter=" ", skip_header=0, names=True)
clus['l2'] = np.nan_to_num(clus['l2'])


In [111]:
n = clus.dtype.names
print n
n.index('isBarrel')
x_train = np.genfromtxt("/Users/innocent/data/clusterShape.csv", delimiter=" ",\
                       skip_header=1, dtype='<f4',
                       usecols=(n.index('isBarrel'), n.index('layer'), n.index('x'),n.index('y'),
#                                n.index('xx'), n.index('yy'), n.index('xy'),
                                n.index('dx'), n.index('dy'), n.index('l2'),
                                n.index('sx'), n.index('sy'), n.index('s'), n.index('q')
                                )
                       )
x_train = np.nan_to_num(x_train)
y_train = np.genfromtxt("/Users/innocent/data/clusterShape.csv", delimiter=" ",\
                       skip_header=1,dtype='<f4',
                       usecols=(n.index('simSX'), n.index('simSY'))
                       )

('isBarrel', 'layer', 'simX', 'simY', 'simSX', 'simSY', 'recX', 'recY', 'x', 'y', 'xx', 'yy', 'xy', 'dx', 'dy', 'l2', 'sx', 'sy', 's', 'q')


In [112]:
print clus.dtype.names
print clus.size

('isBarrel', 'layer', 'simX', 'simY', 'simSX', 'simSY', 'recX', 'recY', 'x', 'y', 'xx', 'yy', 'xy', 'dx', 'dy', 'l2', 'sx', 'sy', 's', 'q')
382361


In [113]:
print x_train.dtype
print x_train.shape
print y_train.dtype.names
print x_train.size
print x_train[0]

print y_train.dtype.names
print y_train.shape
print x_train.size
print y_train[0]

float32
(382361, 11)
None
4205971
[1.00000e+00 1.00000e+00 1.71441e-01 1.18207e+00 9.95017e-01 2.72549e+00
 1.20569e+00 2.00000e+00 3.00000e+00 4.00000e+00 3.44200e+04]
None
(382361, 2)
4205971
[-0.376981  2.08425 ]


In [114]:
weights_ = np.ones(x_train.shape[0])

In [ ]:
#/////Better metrics to monitor while training
#/////////////////////////////////////////////
class Metrics(Callback):
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []
 
    def on_epoch_end(self, epoch, logs={}):
        val_predict = (np.asarray(self.model.predict(self.validation_data[0]))).round()
        val_targ = self.validation_data[1]
        _val_f1 = f1_score(val_targ, val_predict)
        _val_recall = recall_score(val_targ, val_predict)
        _val_precision = precision_score(val_targ, val_predict)
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        print " — val_f1: %f — val_precision: %f — val_recall %f " %(_val_f1, _val_precision, _val_recall)
        return
 
metrics_ = Metrics()

#/////Creating the model
#//////////////////////
dropoutRate_=0.1
reg_rate_=0.0
loss_ = 'mse' #'mse','binary_crossentropy','mae'
Nepoch_=40
batch_size_=512
adam_=optimizers.Adam(lr=0.001)
reduce_lr_ = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=7, min_lr=0.00001)


a_inp = Input(shape=(x_train.shape[1],),name='ins')
a = Dense(256,activation='relu', kernel_initializer='normal',kernel_regularizer=l2(reg_rate_))(a_inp)
a = Dropout(dropoutRate_)(a)
a = Dense(128,activation='relu', kernel_initializer='normal',kernel_regularizer=l2(reg_rate_))(a)
a = Dropout(dropoutRate_)(a)
a = Dense(32,activation='relu', kernel_initializer='normal',kernel_regularizer=l2(reg_rate_))(a)
a = Dropout(dropoutRate_)(a)
a = Dense(16,activation='relu', kernel_initializer='normal',kernel_regularizer=l2(reg_rate_))(a)
a_out = Dense(2, activation='linear', kernel_initializer='normal',name='outs')(a)

model=Model(inputs=a_inp,outputs=a_out)
model.compile(loss=loss_, optimizer=adam_,metrics=['acc'])

In [ ]:
#//////Train the model
#/////////////////////

#NOTE: Due to some unfortunate incompability with the notebooks and keras,
#displaying the progress of training requires additional effort, i.e.
#TQDMNotebookCallback or something similar. This however interferes with
#printing the custom monitoring metrics at the end of epoch. They will
#show correctly after the training finishes.

history=model.fit(x_train,y_train,
        epochs=Nepoch_,
        batch_size=batch_size_,
        sample_weight=weights_,
        callbacks=[reduce_lr_],
#        callbacks=[reduce_lr_,metrics_,TQDMNotebookCallback(metric_format="{name}: {value:0.6f}")],
        validation_split=0.1,
        shuffle=True,
        verbose=1)


Train on 344124 samples, validate on 38237 samples
Epoch 1/40
344124/344124 [==============================] - 3s 10us/step - loss: 2.0371 - acc: 0.7691 - val_loss: 1.1816 - val_acc: 0.7733
Epoch 2/40
344124/344124 [==============================] - 3s 8us/step - loss: 1.1303 - acc: 0.7738 - val_loss: 1.2543 - val_acc: 0.7733
Epoch 3/40
344124/344124 [==============================] - 3s 9us/step - loss: 1.0645 - acc: 0.7739 - val_loss: 1.1742 - val_acc: 0.7729
Epoch 4/40
344124/344124 [==============================] - 4s 11us/step - loss: 1.0395 - acc: 0.7738 - val_loss: 1.3085 - val_acc: 0.7709
Epoch 5/40
344124/344124 [==============================] - 3s 10us/step - loss: 1.0330 - acc: 0.7737 - val_loss: 1.1573 - val_acc: 0.7717
Epoch 6/40
344124/344124 [==============================] - 3s 9us/step - loss: 1.0284 - acc: 0.7737 - val_loss: 1.4009 - val_acc: 0.7686
Epoch 7/40
344124/344124 [==============================] - 3s 9us/step - loss: 1.0227 - acc: 0.7738 - val_loss: 1.450

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.yscale('log')
plt.xlabel('Epoch')
plt.legend(['train loss', 'test loss','test f1'])
plt.show()
plt.savefig('Training_losses.pdf')


In [ ]:
#plt.plot(metrics_.val_f1s)
#plt.plot(metrics_.val_recalls)
#plt.plot(metrics_.val_precisions)
#plt.title('Metrics')
#plt.ylabel('Value')
#plt.xlabel('Epoch')
#plt.legend(['F1','Recall','Precision'])
#plt.savefig('Metrics.pdf')

In [ ]:
y_pred = model.predict(x_train)
print y_pred.shape

In [ ]:
lb = np.ma.masked_where(clus['isBarrel']==0,clus['layer']+1)
le = np.ma.masked_where(clus['isBarrel']==1,clus['layer']+1)
# nn = np.ma.masked_where(np.logical_or(clus['simSY']<5,clus['dy']<clus['simSY']),clus['layer']+1)
plt.scatter(y_pred[:,1],y_train[:,1],s=lb,marker='o',c=clus['layer'])
plt.show()
plt.scatter(y_pred[:,0],y_train[:,0],s=lb,marker='o',c=clus['layer'])
plt.show()
plt.scatter(y_pred[:,1],y_train[:,1],s=le,marker='o',c=clus['layer'])
plt.show()
plt.scatter(y_pred[:,0],y_train[:,0],s=le,marker='o',c=clus['layer'])
plt.show()

In [ ]:
def hist(w) :
    nbins = 100
    y1,xy = np.histogram(y_pred[:,1]-y_train[:,1],np.linspace(-4.,4.,nbins),weights=w, density=True)
    y2,xy = np.histogram(y_pred[:,0]-y_train[:,0],np.linspace(-4.,4.,nbins),weights=w, density=True)
    xy.resize(len(y2))
    return xy,y1,y2

In [ ]:
isS2 = np.equal(clus['sx'],2)*np.greater(clus['x'],0.5)
isl1 = np.equal(clus['layer'],1)
notl1 = np.logical_not(isl1)
notS1 = np.greater(clus['s'],1)
xy,y1,y2 = hist(clus['isBarrel']*notS1) # *isl1)
exy,ey1,ey2 = hist((1-clus['isBarrel'])*notS1)  # *isl1)                                    

In [ ]:
plt.step(xy,y1,where='post')
plt.step(xy,y2,where='post')
plt.show()
plt.step(exy,ey1,where='post')
plt.step(exy,ey2,where='post')
plt.show()